# Assistant-02<br/>Assistant Framework

This notebook showcases a foundational framework to work with Assistants.

### Get an OpenAI client and an Assistant

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import common
import oaihelper as framework

## Get an OpenAI client
client = framework.get_openai_client()

## Create an assistant
assistant = framework.create_assistant(client,"Math Tutor","You are a personal math tutor. Write and run code to answer math questions.", [{"type": "code_interpreter"}], common.gpt_deployment_name)

## Run user messages against the assistant

In [2]:
# --------------------------------------------------------------
# Test assistant
# --------------------------------------------------------------
new_message = framework.generate_response(client,assistant,"What's 2+2", "123", "John")

new_message = framework.generate_response(client,assistant,"What's x*x?", "456", "Sarah")

new_message = framework.generate_response(client,assistant,"What the value of x, for x=m*y+5, if y=2 and m=-10?", "123", "John")

new_message = framework.generate_response(client,assistant,"What is the value of y, for y=x*x?, if x=2?", "456", "Sarah")

Retrieving existing thread for John with wa_id 123
Generated message: 2 + 2 equals 4.
To John: 2 + 2 equals 4.
Retrieving existing thread for Sarah with wa_id 456
Generated message: The expression \( x \times x \) is equivalent to \( x^2 \), which is the square of the variable \( x \).
To Sarah: The expression \( x \times x \) is equivalent to \( x^2 \), which is the square of the variable \( x \).
Retrieving existing thread for John with wa_id 123
Generated message: Given \( y = 2 \) and \( m = -10 \), we can substitute these values into the equation \( x = m \times y + 5 \) to find the value of \( x \).

Let's do the calculation:

\[ x = -10 \times 2 + 5 \]
\[ x = -20 + 5 \]
\[ x = -15 \]

The value of \( x \) is \( -15 \).
To John: Given \( y = 2 \) and \( m = -10 \), we can substitute these values into the equation \( x = m \times y + 5 \) to find the value of \( x \).

Let's do the calculation:

\[ x = -10 \times 2 + 5 \]
\[ x = -20 + 5 \]
\[ x = -15 \]

The value of \( x \) is \(